In [1]:
from HW import *

In [2]:
orbit = HW2AV(S2F('6,5,3/2,4,5/2'), rtype='B',report=1)

Type B_5:
 AV_C(L(6,5,3/2,4,5/2)) 
  = (3, 3, 1, 1, 1, 1, 1) 
 AVC-dim= 30
***
***
*
*
*
*
*


In [308]:
#HW = (frac('3.1'),frac('2.3'),frac('1.1'),frac('-4'),frac('-4.1'),frac('2.5'),frac('1.9'),frac('2'),frac('2.1'),frac('0'))
#HW = (frac('2.5'),)
#HW = (frac('-2'),)
HW = (frac('9/2'),frac('7/2'),frac('5/2'),frac('3/2'),frac('1/2'),)
HW = (frac('1'),frac('4'),frac('3'),frac('2'),frac('1/2'),)
HW = ('3.1','1.1','2.1','-1.9','4.1','2.5')
HW = ('2.3','-4','2','0')
HW = ('2.3','-4','2','0','2.5')
HW = ('3.1','1.1','2.1','-1.9','4.1','-4','2','0','2.5')
HW = ('3.1','1.1','2.1','-1.9','4.1','-4','2','0','2.5','2.3')
HW = ('3.1','2.3','1.1','-4','-4.1','2.5','1.9','2','2.1','0')

orbit = HW2AV(S2F('',rtype = 'C', report = 1)


Type C_10:
 AV_C(L(31/10,23/10,11/10,-4,-41/10,5/2,19/10,2,21/10,0)) 
  = (14, 4, 2) 
 AVC-dim= 192
**************
****
**


In [319]:
orbit = HW2AV(S2F('2,3,4,5,1'),rtype = 'D', report = 1)
orbit = HW2AV(S2F('2,3,4,5,1,1/2,3/2,2.3'),rtype = 'D', report = 1)

Type D_5:
 AV_C(L(2,3,4,5,1)) 
  = (4, 4, 1, 1) 
 AVC-dim= 32
****
****
*
*
Type D_8:
 AV_C(L(2,3,4,5,1,1/2,3/2,23/10)) 
  = (7, 7, 1, 1) 
 AVC-dim= 102
*******
*******
*
*


In [305]:
s = '2, 3, 4, 5, 1'
s.split(',')

['2', ' 3', ' 4', ' 5', ' 1']

In [113]:
HW = (1,1,1,1,1)
HW = (*HW, *(-wt for wt in HW[::-1]))
print(HW)

(1, 1, 1, 1, 1, -1, -1, -1, -1, -1)


In [36]:
print(RSK((1,-1,2,-2,4,3)))
w = (1,1,1,1,1,1,1)
ww = HWM(w)
PP = RSK2part(ww)

print(str_part(PP))

([[-2, 2, 3], [-1, 4], [1]], [[1, 3, 5], [2, 6], [4]])
**************


In [37]:
sw = cell_w2symb(PP)
print(symbol2repn(sw))
ssw = specialsymbol(sw)
print(symbol2repn(ssw))

((7,), ())
((7,), ())


In [67]:
repn2genericdegree(symbol2repn(ssw),'C')

0

In [70]:
jindD2B(((2,4),(3,4)))

((3, 4), (2, 4))

In [75]:
(0-1)//2

-1

In [211]:
hw = (3,4,1,-2,0,-3,5,6)
hwm = HWM(hw)
RSK(hwm)

([[-6, -5, -4, -3], [-3, -1, 0, 2], [-2, 0, 5, 6], [1, 3], [3, 4]],
 [[1, 2, 7, 8], [3, 5, 11, 13], [4, 10, 12, 16], [6, 14], [9, 15]])